In [ ]:


# Input: a classification model, a train table, and a val table

# collect embeddings on the train table and the val table, add the embeddings as new columns to the table

# fit a umap model on the train table, and transform the val table


In [ ]:
import tlc
import timm
from tlc_tools.common import infer_torch_device
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from tlc_tools.embeddings import add_embeddings_to_table


In [ ]:
model_name = "resnet18"

In [ ]:
train_table = tlc.Table.from_names("initial", "CIFAR-10-train", "3LC Tutorials")
val_table = tlc.Table.from_names("initial", "CIFAR-10-val", "3LC Tutorials")

In [ ]:
device = infer_torch_device()
model = timm.create_model(model_name, pretrained=True, num_classes=0)
model = model.to(device)

In [ ]:

transform = Compose([
    Resize(256),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def transformed_image(sample):
    return transform(sample[0])

train_table.clear_maps()
train_table.map(transformed_image)

val_table.clear_maps()
val_table.map(transformed_image)

train_table[0]

In [ ]:
train_table_with_embeddings = add_embeddings_to_table(train_table, model, device=device)

In [ ]:
val_table_with_embeddings = add_embeddings_to_table(val_table, model, device=device)

In [ ]:
url_mapping = tlc.reduce_embeddings_with_producer_consumer(
    val_table_with_embeddings,
    [train_table_with_embeddings],
    method="pacmap",
    n_components=2,
)

In [ ]:
reduced_train_table_url = url_mapping[train_table_with_embeddings.url]
reduced_val_table_url = url_mapping[val_table_with_embeddings.url]

In [ ]:
print(f"Reduced train table url: {reduced_train_table_url}")
print(f"Reduced val table url: {reduced_val_table_url}")